Dependencies

In [ ]:
%pip install rdkit

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#!pip uninstall sklearn-pandas
!pip install scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 32.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


Merge and save

In [ ]:
import pandas as pd
import os

file_names = [
    '/content/drive/MyDrive/Project/Оценка молекул/Linkers/sample_200_benzimidazole_AB.csv',
    '/content/drive/MyDrive/Project/Оценка молекул/Linkers/sample_200_benzimidazole_gyrA.csv',
    '/content/drive/MyDrive/Project/Оценка молекул/Linkers/sample_200_benzimidazole_gyrB.csv'
]

data = []

for file_name in file_names:
    df = pd.read_csv(file_name)

    if not df.empty:
        short_file_name = os.path.basename(file_name)

        df['file name'] = short_file_name

        filtered_rows = []
        for _, row in df.iterrows():
            try:
                int(row['Epoch'])
                filtered_rows.append(row)
            except ValueError:
                pass

        filtered_df = pd.DataFrame(filtered_rows)

        data.append(filtered_df)
    else:
        print(f"Файл {file_name} пуст.")

combined_df = pd.concat(data, ignore_index=True)

combined_df = combined_df.drop_duplicates(subset="Smiles")

combined_df.to_csv('/content/drive/MyDrive/Project/Оценка молекул/Linkers/combined_sample_200_benzimidazole_gyrases.csv', index=False)

Sascore

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import sascorer
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Project/Оценка молекул/Linkers/combined_sample_200_benzimidazole_gyrases.csv')


def check_sascore(smiles):
    """Calculates SAScore for a SMILES string"""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return sascorer.calculateScore(mol)

df['SAScore'] = df['Smiles'].apply(check_sascore)

df.to_csv('/content/drive/MyDrive/Project/Оценка молекул/Linkers/combined_sample_200_benzimidazole_gyrases.csv', index=False)

QED, Lipinski5, Lipinski3

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Project/Оценка молекул/Linkers/combined_sample_200_benzimidazole_gyrases.csv')

import os
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski


def check_qed(mol):
    """Calculates QED-score."""
    return Descriptors.qed(mol)

def check_lipinski_ro5(mol):
    """Checks Lipinski's rule of five for drug-like molecules."""
    return Lipinski.NumHDonors(mol) < 5 and \
           Lipinski.NumHAcceptors(mol) < 10 and \
           Descriptors.MolWt(mol) < 500 and \
           Descriptors.MolLogP(mol) < 5

def check_lipinski_ro3(mol):
    """Checks Lipinski's rule of three for lead-like molecules."""
    return Lipinski.NumHDonors(mol) < 3 and \
           Lipinski.NumHAcceptors(mol) < 3 and \
           Descriptors.MolWt(mol) < 300 and \
           Descriptors.MolLogP(mol) < 3

def process_molecule(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        qed_score = check_qed(mol)
        lipinski_ro5_pass = check_lipinski_ro5(mol)
        lipinski_ro3_pass = check_lipinski_ro3(mol)
        return qed_score, lipinski_ro5_pass, lipinski_ro3_pass
    else:
        return None, None, None

df['QED Score'] = None
df['Lipinski Ro5 Pass'] = None
df['Lipinski Ro3 Pass'] = None

for index, row in df.iterrows():
    smiles = row['Smiles']
    qed_score, lipinski_ro5_pass, lipinski_ro3_pass = process_molecule(smiles)

    df.at[index, 'QED Score'] = qed_score
    df.at[index, 'Lipinski Ro5 Pass'] = lipinski_ro5_pass
    df.at[index, 'Lipinski Ro3 Pass'] = lipinski_ro3_pass

df.to_csv('/content/drive/MyDrive/Project/Оценка молекул/Linkers/combined_sample_200_benzimidazole_gyrases.csv', index=False)
df.head()

Brenk score

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams

def check_brenk_toxicity(df, smiles_column='Smiles'):
    params_brenk = FilterCatalogParams()
    params_brenk.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
    catalog_brenk = FilterCatalog(params_brenk)

    def is_non_toxic(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return False
            if catalog_brenk.HasMatch(mol):
                return False
            return True
        except:
            return False

    df['Brenk Score'] = df[smiles_column].apply(is_non_toxic)

    return df

MIC

In [ ]:
import pandas as pd
import numpy as np
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray

with open("/content/drive/MyDrive/Project/Оценка молекул/Linkers/benz_model_gb.pkl", "rb") as f:
  model = pickle.load(f)

def get_fingerprint(mol):
    """Конвертирует SMILES в отпечаток Morgan размером 256 бит."""
    fp_array = np.zeros((256,), dtype=np.int8)  # Размер массива для отпечатка Morgan
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256)  # Размер отпечатка: 256 бит
    ConvertToNumpyArray(fp, fp_array)
    return fp_array

def predict_mic(smiles, model):
    """Возвращает предсказанный MIC для молекулы."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = get_fingerprint(mol)
    mic = model.predict([fp])
    return mic[0]

df = pd.read_csv("/content/drive/MyDrive/Project/Оценка молекул/Linkers/fragment_mols.csv") # или если датасет уже загружен в df

df['Predicted MIC (log)'] = df['Smiles'].apply(lambda x: predict_mic(x, model))
df['Predicted MIC'] = 10**df['Predicted MIC (log)']
df = df.drop(columns=['Predicted MIC (log)'])

df.to_csv("/content/drive/MyDrive/Project/Оценка молекул/Linkers/fragment_mols.csv", index=False)

Final scoring script

In [ ]:
%pip install rdkit
import pandas as pd
import numpy as np
import os
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Lipinski, FilterCatalog, FilterCatalogParams
import sascorer


with open("/content/drive/MyDrive/Project/Оценка молекул/Linkers/benz_model_gb.pkl", "rb") as f:
    model = pickle.load(f)

def calculate_scores(file_path, output_path):
    df = pd.read_csv(file_path)


    def calculate_sascore(smiles):
        mol = Chem.MolFromSmiles(smiles)
        return sascorer.calculateScore(mol) if mol else None

    def calculate_descriptors(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            qed_score = Descriptors.qed(mol)
            ro5_pass = Lipinski.NumHDonors(mol) < 5 and Lipinski.NumHAcceptors(mol) < 10 and \
                       Descriptors.MolWt(mol) < 500 and Descriptors.MolLogP(mol) < 5
            ro3_pass = Lipinski.NumHDonors(mol) < 3 and Lipinski.NumHAcceptors(mol) < 3 and \
                       Descriptors.MolWt(mol) < 300 and Descriptors.MolLogP(mol) < 3
            return qed_score, ro5_pass, ro3_pass
        return None, None, None

    def calculate_brenk(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            params = FilterCatalogParams()
            params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
            catalog = FilterCatalog(params)
            return not catalog.HasMatch(mol)
        return False

    def predict_mic(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            fp = np.zeros((256,), dtype=np.int8)
            fp_vect = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256)
            AllChem.DataStructs.ConvertToNumpyArray(fp_vect, fp)
            mic_log = model.predict([fp])[0]
            return mic_log, 10 ** mic_log
        return None, None

    df['SAScore'] = df['Smiles'].apply(calculate_sascore)
    df[['QED Score', 'Lipinski Ro5 Pass', 'Lipinski Ro3 Pass']] = df['Smiles'].apply(
        lambda x: pd.Series(calculate_descriptors(x))
    )
    df['Brenk Score'] = df['Smiles'].apply(calculate_brenk)
    df[['Predicted MIC (log)', 'Predicted MIC']] = df['Smiles'].apply(
        lambda x: pd.Series(predict_mic(x))
    )

    df.to_csv(output_path, index=False)
    print(f"Результаты сохранены в {output_path}")